### 제시된 spotify_top_songs_audio_features.csv 파일을 이용하여 streams를 예측하는 모델을 완성하고
### 완성된 모델의 개발과정에서 작성한 코드와 테스트 데이터에 대한 결정계수를 답안으로 작성하시오

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [2]:
df = pd.read_csv("./data/spotify_top_songs_audio_features.csv")
df.head()

,id,artist_names,track_name,source,key,mode,time_signature,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,loudness,tempo,duration_ms,weeks_on_chart,streams
0,000xQL6tZNLJzIrtIgxqSl,"ZAYN, PARTYNEXTDOOR",Still Got Time (feat. PARTYNEXTDOOR),RCA Records Label,G,Major,4 beats,0.748,0.627,0.0639,0.13100,0.0,0.0852,0.524,-6.029,120.963,188491,17,107527761
1,003eoIwxETJujVWmNFMoZy,Alessia Cara,Growing Pains,Def Jam Recordings,C#/Db,Minor,4 beats,0.353,0.755,0.7330,0.08220,0.0,0.3900,0.437,-6.276,191.153,193680,2,9944865
2,003vvx7Niy0yvhvHt4a68B,The Killers,Mr. Brightside,Island Records,C#/Db,Major,4 beats,0.352,0.911,0.0747,0.00121,0.0,0.0995,0.236,-5.230,148.033,222973,125,512388123
3,00B7TZ0Xawar6NZ00JFomN,"Cardi B, Chance the Rapper",Best Life (feat. Chance The Rapper),Atlantic/KSR,A,Major,4 beats,0.620,0.625,0.5530,0.28700,0.0,0.3140,0.665,-7.438,167.911,284856,2,11985346
4,00Blm7zeNqgYLPtW6zg8cj,"Post Malone, The Weeknd",One Right Now (with The Weeknd),Republic Records,C#/Db,Major,4 beats,0.687,0.781,0.0530,0.03610,0.0,0.0755,0.688,-4.806,97.014,193507,30,301860377


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6513 entries, 0 to 6512
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                6513 non-null   object 
 1   artist_names      6513 non-null   object 
 2   track_name        6513 non-null   object 
 3   source            6513 non-null   object 
 4   key               6513 non-null   object 
 5   mode              6513 non-null   object 
 6   time_signature    6513 non-null   object 
 7   danceability      6513 non-null   float64
 8   energy            6513 non-null   float64
 9   speechiness       6513 non-null   float64
 10  acousticness      6513 non-null   float64
 11  instrumentalness  6513 non-null   float64
 12  liveness          6513 non-null   float64
 13  valence           6513 non-null   float64
 14  loudness          6513 non-null   float64
 15  tempo             6513 non-null   float64
 16  duration_ms       6513 non-null   int64  


In [4]:
df.describe()

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,loudness,tempo,duration_ms,weeks_on_chart,streams
count,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6513.000000,6.513000e+03
mean,0.681731,0.636522,0.121933,0.236761,0.012469,0.180168,0.492412,-6.350667,122.117244,202566.684170,13.614924,1.089647e+08
std,0.141787,0.164813,0.113441,0.244784,0.075151,0.138054,0.227001,2.536114,29.416097,49199.592333,24.084051,2.276695e+08
min,0.150000,0.021800,0.023200,0.000008,0.000000,0.019700,0.032000,-34.475000,46.718000,30133.000000,1.000000,2.525159e+06
25%,0.591000,0.534000,0.044000,0.044400,0.000000,0.097400,0.316000,-7.564000,98.007000,173038.000000,1.000000,8.695060e+06
50%,0.698000,0.651000,0.072200,0.145000,0.000000,0.124000,0.489000,-5.983000,120.034000,198367.000000,4.000000,2.712856e+07
75%,0.785000,0.759000,0.163000,0.356000,0.000041,0.219000,0.669000,-4.673000,142.025000,226003.000000,16.000000,9.951456e+07
max,0.985000,0.989000,0.966000,0.994000,0.953000,0.977000,0.982000,1.509000,212.117000,690732.000000,367.000000,3.528450e+09


In [5]:
df.shape

(6513, 19)

In [6]:
ndf = df.drop(["artist_names", "id", "track_name", "source", "key", "mode"], axis = 1)

In [7]:
ndf["time_signature"] = ndf["time_signature"].str.slice(stop = 1).astype(int)

In [8]:
ndf.dtypes

time_signature        int32
danceability        float64
energy              float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
loudness            float64
tempo               float64
duration_ms           int64
weeks_on_chart        int64
streams               int64
dtype: object

In [9]:
ss = StandardScaler()

In [11]:
ndf_ss = ss.fit_transform(ndf.iloc[:, :-1])

In [12]:
ndf_ss = pd.DataFrame(ndf_ss, columns = ndf.iloc[:, :-1].columns)

In [13]:
ndf_ss = pd.concat([ndf_ss, ndf.iloc[:, -1]], axis = 1)

In [14]:
ndf_ss.head()

,time_signature,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,loudness,tempo,duration_ms,weeks_on_chart,streams
0,0.142698,0.467421,-0.057781,-0.511608,-0.432091,-0.165928,-0.687957,0.139166,0.126844,-0.039242,-0.286115,0.140563,107527761
1,0.142698,-2.318656,0.718919,5.387041,-0.631465,-0.165928,1.520039,-0.244123,0.029444,2.347050,-0.180639,-0.482303,9944865
2,0.142698,-2.325709,1.665521,-0.416398,-0.962353,-0.165928,-0.584366,-1.129651,0.441917,0.881074,0.414798,4.625203,512388123
3,0.142698,-0.435409,-0.069917,3.800198,0.205254,-0.165928,0.969489,0.760357,-0.428773,1.556878,1.672689,-0.482303,11985346
4,0.142698,0.037166,0.876686,-0.607700,-0.819809,-0.165928,-0.758224,0.861686,0.609115,-0.853450,-0.184156,0.680381,301860377
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6508,0.142698,-0.470676,0.864550,0.115195,-0.695608,-0.165928,-0.182320,-0.173633,-0.040748,1.287923,0.114467,-0.523828,17583532
6509,0.142698,0.601435,-1.963122,-0.371437,2.456387,-0.165928,-0.544524,1.200919,-2.250991,-1.344104,0.169533,-0.482303,10526635
6510,0.142698,1.708813,0.567220,1.340591,-0.752806,-0.165928,-0.486571,1.566584,0.088988,-0.785591,0.693134,-0.523828,5404939
6511,-3.040150,0.319301,-0.646374,1.675591,0.188911,-0.165561,1.534527,-1.032727,-0.893299,1.764739,0.168191,-0.523828,12872010


In [15]:
x = sm.add_constant(ndf_ss.drop("streams", axis = 1))
y = ndf_ss["streams"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 7)

In [17]:
lr = LinearRegression()
lr.fit(x_train, y_train)

In [18]:
lr.fit(x_train, y_train)

LinearRegression()

In [19]:
lr.score(x_test, y_test)

0.7236107480918562

In [20]:
model = sm.OLS(y_train, x_train)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     1129.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4871   BIC:                         1.955e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.098e+08   1.69e+06     65.122      0.000    1.06e+08    1.13e+08
time_signature   -2.492e+06    1.7e+06     -1.464      0.143   -5.83e+06    8.46e+05
danceability      3.384e+05   1.94e+06      0.174      0.862   -3.47e+06    4.14e+06
energy             5.45e+05   2.76e+06      0.197      0.844   -4.87e+06    5.96e+06
speechiness      -3.579e+06   1.74e+06     -2.061      0.039   -6.98e+06   -1.74e+05
acousticness      1.951e+06   2.07e+06      0.943      0.346   -2.11e+06    6.01e+06
instrumentalness -6.582e+05   1.64e+06     -0.400      0.689   -3.88e+06    2.56e+06
liveness          8.305e+05    1.7e+06      0.488      0.625    -2.5e+06    4.16e+06
valence            5.43e+05      2e+06      0.271      0.786   -3.38e+06    4.47e+06
loudness         -2.403e+06   2.53e+06     -0.949      0.343   -7.36e+06    2.56e+06
tempo             4.923e+05   1.72e+06      0.285      0.775   -2.89e+06    3.87e+06
duration_ms      -2.195e+06   1.71e+06     -1.283      0.200   -5.55e+06    1.16e+06
weeks_on_chart     1.98e+08   1.72e+06    115.280      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3464.351   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351694.219
Skew:                           2.593   Prob(JB):                         0.00
Kurtosis:                      44.247   Cond. No.                         3.27
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
vif = pd.DataFrame()
vif["VIF_Factor"] = [variance_inflation_factor(x_train.values, i) for i in range(x_train.shape[1])]
vif["feature"] = x_train.columns
vif.round(1)

,VIF_Factor,feature
0,1.0,const
1,1.0,time_signature
2,1.3,danceability
3,2.7,energy
4,1.1,speechiness
5,1.5,acousticness
6,1.1,instrumentalness
7,1.0,liveness
8,1.4,valence
9,2.2,loudness


In [22]:
new_x = x_train.drop("danceability", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     1232.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4872   BIC:                         1.955e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.098e+08   1.69e+06     65.134      0.000    1.07e+08    1.13e+08
time_signature   -2.469e+06    1.7e+06     -1.455      0.146    -5.8e+06    8.58e+05
energy            4.684e+05   2.73e+06      0.172      0.864   -4.88e+06    5.81e+06
speechiness      -3.522e+06   1.71e+06     -2.065      0.039   -6.87e+06   -1.78e+05
acousticness      1.862e+06   2.01e+06      0.928      0.353   -2.07e+06     5.8e+06
instrumentalness -6.622e+05   1.64e+06     -0.403      0.687   -3.88e+06    2.56e+06
liveness          7.959e+05   1.69e+06      0.471      0.637   -2.51e+06    4.11e+06
valence           6.586e+05   1.89e+06      0.348      0.728   -3.05e+06    4.36e+06
loudness         -2.353e+06   2.51e+06     -0.936      0.349   -7.28e+06    2.58e+06
tempo             4.597e+05   1.71e+06      0.268      0.789    -2.9e+06    3.82e+06
duration_ms      -2.228e+06    1.7e+06     -1.311      0.190   -5.56e+06     1.1e+06
weeks_on_chart     1.98e+08   1.72e+06    115.297      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3463.731   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351482.686
Skew:                           2.593   Prob(JB):                         0.00
Kurtosis:                      44.235   Cond. No.                         3.10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
new_x = new_x.drop("energy", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     1356.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4873   BIC:                         1.955e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.098e+08   1.69e+06     65.140      0.000    1.07e+08    1.13e+08
time_signature   -2.456e+06    1.7e+06     -1.449      0.147   -5.78e+06    8.68e+05
speechiness      -3.527e+06   1.71e+06     -2.068      0.039   -6.87e+06   -1.83e+05
acousticness      1.745e+06   1.89e+06      0.925      0.355   -1.95e+06    5.45e+06
instrumentalness -6.326e+05   1.63e+06     -0.387      0.699   -3.84e+06    2.57e+06
liveness           8.37e+05   1.67e+06      0.501      0.617   -2.44e+06    4.11e+06
valence            7.48e+05   1.82e+06      0.412      0.681   -2.81e+06    4.31e+06
loudness         -2.089e+06   1.99e+06     -1.049      0.294   -5.99e+06    1.81e+06
tempo             4.704e+05   1.71e+06      0.275      0.784   -2.89e+06    3.83e+06
duration_ms      -2.216e+06    1.7e+06     -1.305      0.192   -5.54e+06    1.11e+06
weeks_on_chart     1.98e+08   1.72e+06    115.395      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3464.497   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351701.295
Skew:                           2.594   Prob(JB):                         0.00
Kurtosis:                      44.247   Cond. No.                         1.87
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
new_x = new_x.drop("tempo", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     1507.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4874   BIC:                         1.955e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.098e+08   1.69e+06     65.148      0.000    1.07e+08    1.13e+08
time_signature   -2.469e+06   1.69e+06     -1.457      0.145   -5.79e+06    8.53e+05
speechiness      -3.466e+06   1.69e+06     -2.050      0.040   -6.78e+06   -1.51e+05
acousticness      1.722e+06   1.89e+06      0.913      0.361   -1.97e+06    5.42e+06
instrumentalness -6.298e+05   1.63e+06     -0.385      0.700   -3.83e+06    2.57e+06
liveness          8.283e+05   1.67e+06      0.496      0.620   -2.45e+06     4.1e+06
valence           7.697e+05   1.82e+06      0.424      0.672   -2.79e+06    4.33e+06
loudness         -2.063e+06   1.99e+06     -1.038      0.299   -5.96e+06    1.83e+06
duration_ms      -2.218e+06    1.7e+06     -1.307      0.191   -5.55e+06    1.11e+06
weeks_on_chart     1.98e+08   1.72e+06    115.405      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3465.658   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           352068.442
Skew:                           2.595   Prob(JB):                         0.00
Kurtosis:                      44.269   Cond. No.                         1.86
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
new_x = new_x.drop("instrumentalness", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     1695.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4875   BIC:                         1.955e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.098e+08   1.69e+06     65.153      0.000    1.06e+08    1.13e+08
time_signature -2.471e+06   1.69e+06     -1.458      0.145   -5.79e+06    8.51e+05
speechiness    -3.424e+06   1.69e+06     -2.029      0.043   -6.73e+06   -1.16e+05
acousticness    1.733e+06   1.88e+06      0.919      0.358   -1.96e+06    5.43e+06
liveness        8.214e+05   1.67e+06      0.492      0.623   -2.45e+06     4.1e+06
valence         8.235e+05   1.81e+06      0.455      0.649   -2.72e+06    4.37e+06
loudness       -1.961e+06   1.97e+06     -0.995      0.320   -5.82e+06     1.9e+06
duration_ms    -2.189e+06    1.7e+06     -1.290      0.197   -5.51e+06    1.14e+06
weeks_on_chart   1.98e+08   1.72e+06    115.435      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3465.078   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           352029.992
Skew:                           2.594   Prob(JB):                         0.00
Kurtosis:                      44.267   Cond. No.                         1.81
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
new_x = new_x.drop("valence", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     1938.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4876   BIC:                         1.954e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.098e+08   1.69e+06     65.157      0.000    1.06e+08    1.13e+08
time_signature -2.454e+06   1.69e+06     -1.449      0.147   -5.77e+06    8.67e+05
speechiness    -3.382e+06   1.68e+06     -2.008      0.045   -6.69e+06   -7.93e+04
acousticness    1.784e+06   1.88e+06      0.948      0.343    -1.9e+06    5.47e+06
liveness        8.319e+05   1.67e+06      0.498      0.618   -2.44e+06    4.11e+06
loudness       -1.691e+06   1.88e+06     -0.900      0.368   -5.37e+06    1.99e+06
duration_ms    -2.312e+06   1.67e+06     -1.381      0.167   -5.59e+06     9.7e+05
weeks_on_chart  1.981e+08   1.71e+06    115.561      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3465.107   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351875.081
Skew:                           2.594   Prob(JB):                         0.00
Kurtosis:                      44.258   Cond. No.                         1.64
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
new_x = new_x.drop("liveness", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     2261.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4877   BIC:                         1.954e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.098e+08   1.68e+06     65.166      0.000    1.06e+08    1.13e+08
time_signature -2.468e+06   1.69e+06     -1.457      0.145   -5.79e+06    8.52e+05
speechiness    -3.334e+06   1.68e+06     -1.982      0.047   -6.63e+06   -3.69e+04
acousticness    1.775e+06   1.88e+06      0.944      0.345   -1.91e+06    5.46e+06
loudness       -1.636e+06   1.88e+06     -0.872      0.383   -5.31e+06    2.04e+06
duration_ms    -2.312e+06   1.67e+06     -1.381      0.167   -5.59e+06    9.69e+05
weeks_on_chart   1.98e+08   1.71e+06    115.679      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3465.695   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           352054.661
Skew:                           2.595   Prob(JB):                         0.00
Kurtosis:                      44.268   Cond. No.                         1.63
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
new_x = new_x.drop("loudness", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     2713.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97690.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4878   BIC:                         1.954e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.098e+08   1.68e+06     65.164      0.000    1.06e+08    1.13e+08
time_signature   -2.5e+06   1.69e+06     -1.477      0.140   -5.82e+06    8.19e+05
speechiness    -3.165e+06   1.67e+06     -1.895      0.058   -6.44e+06     1.1e+05
acousticness     2.47e+06    1.7e+06      1.449      0.147   -8.72e+05    5.81e+06
duration_ms    -2.317e+06   1.67e+06     -1.384      0.166    -5.6e+06    9.65e+05
weeks_on_chart  1.979e+08   1.71e+06    115.768      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3464.474   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351867.496
Skew:                           2.593   Prob(JB):                         0.00
Kurtosis:                      44.257   Cond. No.                         1.19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
new_x = new_x.drop("duration_ms", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     3390.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97691.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4879   BIC:                         1.954e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.098e+08   1.68e+06     65.156      0.000    1.06e+08    1.13e+08
time_signature -2.477e+06   1.69e+06     -1.463      0.144    -5.8e+06    8.42e+05
speechiness    -3.081e+06   1.67e+06     -1.845      0.065   -6.35e+06    1.93e+05
acousticness    2.578e+06    1.7e+06      1.514      0.130    -7.6e+05    5.92e+06
weeks_on_chart  1.979e+08   1.71e+06    115.757      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3469.551   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           352606.568
Skew:                           2.600   Prob(JB):                         0.00
Kurtosis:                      44.300   Cond. No.                         1.19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
new_x = new_x.drop("time_signature", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     4518.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97692.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4880   BIC:                         1.954e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.098e+08   1.69e+06     65.146      0.000    1.06e+08    1.13e+08
speechiness    -3.272e+06   1.66e+06     -1.965      0.049   -6.54e+06   -7736.821
acousticness    2.888e+06   1.69e+06      1.709      0.088   -4.25e+05     6.2e+06
weeks_on_chart  1.979e+08   1.71e+06    115.756      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3467.757   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351570.155
Skew:                           2.598   Prob(JB):                         0.00
Kurtosis:                      44.239   Cond. No.                         1.13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
new_x = new_x.drop("acousticness", axis = 1)
new_x = sm.add_constant(new_x)
model = sm.OLS(y_train, new_x)
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                streams   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     6773.
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:15:38   Log-Likelihood:                -97694.
No. Observations:                4884   AIC:                         1.954e+05
Df Residuals:                    4881   BIC:                         1.954e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.098e+08   1.69e+06     65.134      0.000    1.06e+08    1.13e+08
speechiness    -3.487e+06   1.66e+06     -2.100      0.036   -6.74e+06   -2.32e+05
weeks_on_chart  1.979e+08   1.71e+06    115.722      0.000    1.95e+08    2.01e+08
==============================================================================
Omnibus:                     3466.148   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351396.655
Skew:                           2.596   Prob(JB):                         0.00
Kurtosis:                      44.229   Cond. No.                         1.10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
fin_train = x_train[["const", "speechiness", "weeks_on_chart"]]
fin_test = x_test[["const", "speechiness", "weeks_on_chart"]]

In [33]:
lr.fit(fin_train, y_train)

LinearRegression()

In [34]:
lr.score(fin_test, y_test)

0.7238417989047905